In [ ]:
import os
import math
import pandas as pd
import numpy as np
from scipy.signal import medfilt
from scipy.interpolate import InterpolatedUnivariateSpline

<span style = 'color:red;font-size:25px'>Extract data for the same MMSI.

In [ ]:
input_files = [
                             # File path
]

output_folder = r"  "          # Output directory path

os.makedirs(output_folder, exist_ok=True)
mmsi_data = {}

# Iterate over each input file and group data by MMSI
for file in input_files:
    try:
        df = pd.read_csv(file)
        if 'MMSI' not in df.columns:
            print(f"File {file} does not contain an MMSI column; skipping...")
            continue  
            
        for mmsi, group in df.groupby('MMSI'):
            if mmsi not in mmsi_data:
                mmsi_data[mmsi] = group
            else:
                mmsi_data[mmsi] = pd.concat([mmsi_data[mmsi], group], ignore_index=True)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Save each MMSI’s data to a separate CSV file
for mmsi, data in mmsi_data.items():
    output_file = os.path.join(output_folder, f"{mmsi}.csv")
    data.to_csv(output_file, index=False)

print("Data extraction and saving complete!")

<span style = 'color:red;font-size:25px'>Remove outlier points

In [ ]:
folder_path = r"  "   # Folder path
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# Iterate over each CSV file
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    data = pd.read_csv(file_path)
    if 'BaseDateTime' in data.columns:
        data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')
        data.sort_values('BaseDateTime', inplace=True)
    
    data = data[(data['SOG'] >= 0.5) & (data['SOG'] <= 30)]
    data = data[(data['COG'] >= 0) & (data['COG'] <= 360)]
    data = data[(data['LAT'] >= -90) & (data['LAT'] <= 90)]
    data = data[(data['LON'] >= 0) & (data['LON'] <= 180)]
    data = data[(data['Heading'] >= 0) & (data['Heading'] <= 180)]
    
    # Overwrite the original CSV file
    data.to_csv(file_path, index=False)

print("Data processing complete.")


<span style = 'color:red;font-size:25px'>Fill static data

In [ ]:
# Define target columns
target_columns = ['VesselType', 'Width', 'Length', 'Draft']
folder_path = r"   "
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Iterate over each CSV file in the folder
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    valid_values = {}
    
    for column in target_columns:
        if column in df.columns:
            valid_value = df[column][(df[column].notna()) & ~(df[column].isin(['undefined', 'Undefined', '']))]
            if not valid_value.empty:
                valid_values[column] = valid_value.iloc[0]
            else:
                valid_values[column] = None  

    # Fill target columns with the extracted valid values
    for column in target_columns:
        if column in valid_values and valid_values[column] is not None:
            df[column] = valid_values[column]
    
    df.to_csv(file_path, index=False)
    print(f"Target columns have been filled and saved to {file_path}")

print("All CSV files have been processed.")

<span style = 'color:red;font-size:25px'>Convert timestamp types

In [ ]:
folder_path = "  "  # Folder path
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# Iterate over all CSV files
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    data = pd.read_csv(file_path)
    
    if 'BaseDateTime' in data.columns:
        try:
            data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], format='%Y/%m/%d %H:%M:%S', errors='raise')
        except ValueError:
            data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], errors='coerce')
            data['BaseDateTime'] = data['BaseDateTime'].dt.strftime('%Y/%m/%d %H:%M:%S')
        data.to_csv(file_path, index=False)

print("Timestamps in the CSV files have been updated as needed.")


<span style = 'color:red;font-size:25px'>Split the data

In [ ]:
folder_path = "  "    # Folder path
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# Iterate over all CSV files
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    data = pd.read_csv(file_path)
    segment_start = 0
    segments = []

    # Traverse the data and compute time differences between adjacent records
    for i in range(1, len(data)):
        timestamp1 = pd.to_datetime(data['BaseDateTime'].iloc[i - 1], format='%Y/%m/%d %H:%M:%S')
        timestamp2 = pd.to_datetime(data['BaseDateTime'].iloc[i], format='%Y/%m/%d %H:%M:%S')
        
        if pd.isna(timestamp1) or pd.isna(timestamp2):
            continue 
            
        timestamp_diff = (timestamp2 - timestamp1).total_seconds() / 3600 

        # If the time gap between two adjacent records exceeds 6 hours, split the data
        if timestamp_diff > 6:
            segment_data = data.iloc[segment_start:i]
            segments.append(segment_data)
            segment_start = i

    if len(segments) > 0:
        
        # Save each segment to a new CSV file
        for idx, segment_data in enumerate(segments):
            mmsi_number = data['MMSI'].iloc[0] if 'MMSI' in data.columns else 'Unknown'
            segment_filename = f"{mmsi_number}_{idx + 1}.csv"
            segment_path = os.path.join(folder_path, segment_filename)
            segment_data.to_csv(segment_path, index=False)

        # Remove from the original file the rows with '# Timestamp' values that were saved
        for segment in segments:
            data = data[~data['BaseDateTime'].isin(segment['BaseDateTime'])]

        data.to_csv(file_path, index=False)

print("CSV data have been segmented by # Timestamp; the segments were saved to new files and removed from the original files," 
       "and rows with matching '# Timestamp' values were deleted.")

<span style = 'color:red;font-size:25px'>Remove outlier points and duplicate-timestamp points

In [ ]:
def get_distance(lon1, lat1, lon2, lat2):
    """
    Compute the distance between two lon/lat points (km)
    param lon1: longitude of point A
    param lat1: latitude of point A
    param lon2: longitude of point B
    param lat2: latitude of point B
    return: distance (km)
    """
    EARTH_RADIUS = 6371  # Earth radius
    lon1, lat1, lon2, lat2 = map(math.radians, [float(lon1), float(lat1), float(lon2), float(lat2)])
    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a = math.sin(d_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    distance = c * EARTH_RADIUS
    return distance

def clear_list(gjd_list=None, max_speed=56, time_interval=6):
    """
    Remove duplicate timestamps and outlier points
    Delete records with duplicate timestamps, keeping the first occurrence
    Remove outliers: if the distance/time gap between adjacent points exceeds a threshold, drop the abnormal point
    param time_interval: time-splitting threshold; if the gap between adjacent points exceeds this value (here 6 h), split the trajectory into segments
    param max_speed: normal-speed threshold; if the average speed between adjacent points exceeds this value (here 56 km/h), remove the abnormal point
    param gjd_list: trajectory point list
    return: cleaned index list
    """
    if not gjd_list:
        return 

    timestamp_list = []
    repeat_timestamp = []
    for gjd_index, gjd in enumerate(gjd_list):
        gjd_timestamp = gjd[0] 
        if gjd_timestamp in timestamp_list:
            repeat_timestamp.append(gjd_index)  # Log duplicate indices
        else:
            timestamp_list.append(gjd_timestamp)

    valid_indices = [i for i in range(len(gjd_list)) if i not in repeat_timestamp]

    # Handle outlier points
    final_indices = []  
    for idx in valid_indices:
        if idx + 1 >= len(gjd_list): 
            final_indices.append(idx)
            break
        current_point = gjd_list[idx]  
        next_point = gjd_list[idx + 1]  
        lon1, lat1, timestamp1 = float(current_point[2]), float(current_point[1]), pd.to_datetime(current_point[0])
        lon2, lat2, timestamp2 = float(next_point[2]), float(next_point[1]), pd.to_datetime(next_point[0])
        distance = get_distance(lon1, lat1, lon2, lat2)  
        time_diff = (timestamp2 - timestamp1).total_seconds() / 3600  

        # Check whether a point is an outlier
        if time_diff == 0 or (distance / time_diff) > max_speed:
            continue  
        else:
            final_indices.append(idx)
    return final_indices

target_folder = r" "  # Folder path
csv_files = [file for file in os.listdir(target_folder) if file.endswith(".csv")]

for csv_file in csv_files:
    file_path = os.path.join(target_folder, csv_file)
    try:
        data = pd.read_csv(file_path)
        gjd_list = data[['BaseDateTime', 'LAT', 'LON']].values.tolist()
        valid_indices = clear_list(gjd_list)
        cleaned_data = data.iloc[valid_indices]
        cleaned_data.to_csv(file_path, index=False)

    except Exception as e:
        print(f"Error processing file {csv_file}: {e}")

print("All files have been processed.")

<span style = 'color:red;font-size:25px'>Extract records with lat/lon outside a specified range

In [ ]:
# Define the lat/lon range (convert degrees–minutes–seconds to decimal degrees)
def dms_to_decimal(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600

# Latitude range
lat_max = dms_to_decimal(27, 37, 19.793)    # 27°37′19.793″
lat_min = dms_to_decimal(25, 2, 57.534)     # 25°2′57.534″

# Longitude range
lon_max = dms_to_decimal(80, 30, 8.39)      # 80°30′8.39″
lon_min = dms_to_decimal(78, 16, 39.534)    # 78°16′39.534″
target_folder = r"  "   # Folder path

outside_range_files = []

csv_files = [file for file in os.listdir(target_folder) if file.endswith(".csv")]

# Iterate over each CSV file
for csv_file in csv_files:
    file_path = os.path.join(target_folder, csv_file)
    try:
        data = pd.read_csv(file_path, usecols=['LAT', 'LON'])
        data = data.dropna(subset=['LAT', 'LON'])
        data['LAT'] = pd.to_numeric(data['LAT'], errors='coerce')
        data['LON'] = pd.to_numeric(data['LON'], errors='coerce')
        data = data.dropna(subset=['LAT', 'LON'])
        lat_out_of_range = data[(data['LAT'] < lat_min) | (data['LAT'] > lat_max)]
        is_lat_out = not lat_out_of_range.empty
        lon_out_of_range = data[(data['LON'] < lon_min) | (data['LON'] > lon_max)]
        is_lon_out = not lon_out_of_range.empty

        # Record the filename, out-of-range fields, and the exact values as needed
        if is_lat_out or is_lon_out:
            status = ""
            details = ""
            max_display = 5 
            if is_lat_out and is_lon_out:
                status = "Latitude and Longitude out of range"
                lat_values = lat_out_of_range['LAT'].values[:max_display]
                lon_values = lon_out_of_range['LON'].values[:max_display]
                details += f"Latitude out of range values (showing up to {max_display}):\n{lat_values}\n"
                details += f"Longitude out of range values (showing up to {max_display}):\n{lon_values}\n"
            elif is_lat_out:
                status = "Latitude out of range"
                lat_values = lat_out_of_range['LAT'].values[:max_display]
                details += f"Latitude out of range values (showing up to {max_display}):\n{lat_values}\n"
            else:
                status = "Longitude out of range"
                lon_values = lon_out_of_range['LON'].values[:max_display]
                details += f"Longitude out of range values (showing up to {max_display}):\n{lon_values}\n"
            outside_range_files.append((csv_file, status, details))
    except Exception as e:
        print(f"Error reading file {csv_file}: {e}")

# Output all matching CSV filenames, the out-of-range cases, and the corresponding values
if outside_range_files:
    print("The following files contain values outside the specified lat/lon range:")
    for filename, status, details in outside_range_files:
        print(f"\nFilename: {filename}")
        print(f"Out-of-range case: {status}")
        print(f"Out-of-range values:\n{details}")
else:
    print("No files were found with values outside the specified lat/lon range.")

# <span style = 'color:red;font-size:25px'>Interpolate the data

In [ ]:
input_folder = r"   "   # Folder path

csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

for csv_file in csv_files:
    file_path = os.path.join(input_folder, csv_file)
    data = pd.read_csv(file_path)
    data = data.drop_duplicates(subset='BaseDateTime', keep='first')
    data = data.sort_values(by='BaseDateTime')
    data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], format='%Y/%m/%d %H:%M:%S')
    start_time = data['BaseDateTime'].min()
    end_time = data['BaseDateTime'].max()
    new_timestamps = pd.date_range(start=start_time, end=end_time, freq='1min')  # 修改为 '1min'
    data['Timestamp_numeric'] = data['BaseDateTime'].astype(np.int64) // 10**9
    new_timestamps_numeric = new_timestamps.astype(np.int64) // 10**9

    # Ensure there are enough data points for interpolation
    if len(data) < 2:
        continue 

    # Interpolate the 'Latitude' and 'Longitude' columns
    try:
        valid_data = data.dropna(subset=['LAT', 'LON'])
        if len(valid_data) < 4:
            continue  
        new_latitudes = []
        new_longitudes = []
        
        for i in range(len(new_timestamps_numeric)):
            timestamp = new_timestamps_numeric[i]
            prev_idx = np.searchsorted(valid_data['Timestamp_numeric'], timestamp, side='right') - 1
            next_idx = prev_idx + 1

            # Ensure there are both previous and next points
            if prev_idx >= 0 and next_idx < len(valid_data):
                prev_timestamp = valid_data.iloc[prev_idx]['Timestamp_numeric']
                next_timestamp = valid_data.iloc[next_idx]['Timestamp_numeric']
                time_diff = next_timestamp - prev_timestamp
                if time_diff <= 120:  
                    spline_lat = InterpolatedUnivariateSpline(valid_data['Timestamp_numeric'], valid_data['LAT'])
                    spline_lon = InterpolatedUnivariateSpline(valid_data['Timestamp_numeric'], valid_data['LON'])
                    new_latitudes.append(spline_lat(timestamp))
                    new_longitudes.append(spline_lon(timestamp))
                else:  
                    lat_interp = np.interp(timestamp, valid_data['Timestamp_numeric'], valid_data['LAT'])
                    lon_interp = np.interp(timestamp, valid_data['Timestamp_numeric'], valid_data['LON'])
                    new_latitudes.append(lat_interp)
                    new_longitudes.append(lon_interp)
            else:
                new_latitudes.append(np.nan)
                new_longitudes.append(np.nan)

    except Exception as e:
        print(f"Error interpolating 'Latitude' and 'Longitude' in file {csv_file}: {e}")
        continue

    # Create a new DataFrame with the interpolated data
    interpolated_data = pd.DataFrame({
        'BaseDateTime': new_timestamps,
        'LAT': new_latitudes,
        'LON': new_longitudes
    })
    interpolated_data['BaseDateTime'] = interpolated_data['BaseDateTime'].apply(lambda x: f"{x.year}/{x.month}/{x.day} {x.hour}:{x.minute}:{x.second}")
    interpolated_data['Timestamp_numeric'] = pd.to_datetime(interpolated_data['BaseDateTime'], format='%Y/%m/%d %H:%M:%S').astype(np.int64) // 10**9

    # Add fixed columns: 'MMSI', 'Ship type', 'Width', 'Length', 'Draught'
    fixed_columns = ['MMSI', 'VesselType', 'Width', 'Length', 'Draft']
    for col in fixed_columns:
        if col in data.columns:
            value = data[col].iloc[0] 
            interpolated_data[col] = value
        else:
            interpolated_data[col] = np.nan  

    # Compute the mean over the previous and next 3–5 values for 'SOG', 'COG', and 'Heading'
    for col in ['SOG', 'COG', 'Heading']:
        if col in data.columns:
            data_col = data[['Timestamp_numeric', col]].dropna()
            data_col = data_col.sort_values('Timestamp_numeric').reset_index(drop=True)
            averages = []
            timestamps_numeric = interpolated_data['Timestamp_numeric'].values
            for timestamp in timestamps_numeric:
                idx = np.searchsorted(data_col['Timestamp_numeric'], timestamp)
                start_idx = max(0, idx - 5)
                end_idx = min(len(data_col), idx + 5)
                window_data = data_col.iloc[start_idx:end_idx][col]
                if len(window_data) >= 1:
                    avg_value = window_data.mean()
                    averages.append(avg_value)
                else:
                    averages.append(np.nan)
            interpolated_data[col] = averages
        else:
            interpolated_data[col] = np.nan 

    interpolated_data.drop(columns=['Timestamp_numeric'], inplace=True)
    interpolated_data.to_csv(file_path, index=False)
    
    print(f"File {csv_file} has been processed and overwritten.")

print("All files have been processed.")

<span style = 'color:red;font-size:25px'>Median filtering

In [ ]:
input_folder = r"   "   # Folder path
csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

for csv_file in csv_files:
    try:
        file_path = os.path.join(input_folder, csv_file)
        data = pd.read_csv(file_path)
        data = data.sort_values(by='BaseDateTime').reset_index(drop=True)
        data = data.dropna(subset=['LAT', 'LON']).reset_index(drop=True)
        window_size = 5

        # Check whether the data length is sufficient to apply filtering
        if len(data) >= window_size:
            data_to_filter = data[['LAT', 'LON']].copy()
            data_filtered_lat = medfilt(data_to_filter['LAT'], kernel_size=window_size)
            data_filtered_lon = medfilt(data_to_filter['LON'], kernel_size=window_size)
            data.loc[2:, 'LAT'] = data_filtered_lat[2:]
            data.loc[2:, 'LON'] = data_filtered_lon[2:]
        else:
            print(f"File {csv_file} has fewer data points than the window size; skipping filtering.")
        data.to_csv(file_path, index=False)
        print(f"File {csv_file} has been processed and overwritten.")
    except Exception as e:
        print(f"Error processing file {csv_file}: {e}")
        continue

print("All files have been processed.")

<span style = 'color:red;font-size:25px'>Extract points with the same timestamp

In [ ]:
input_folder = r"  "   # Folder path

csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]
all_data = pd.DataFrame()

# Read and merge data from each CSV file
for csv_file in csv_files:
    file_path = os.path.join(input_folder, csv_file)
    data = pd.read_csv(file_path)
    required_columns = ['# Timestamp', 'MMSI', 'Latitude', 'Longitude', 'SOG', 'COG', 'Heading']
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        print(f"File {csv_file} is missing columns {missing_columns}; skipping this file.")
        continue

    data['# Timestamp'] = pd.to_datetime(data['# Timestamp'], format='%Y/%m/%d %H:%M:%S', errors='coerce')
    data = data.dropna(subset=['# Timestamp'])
    data['Timestamp_min'] = data['# Timestamp'].dt.floor('min')  
    data = data[['MMSI', 'Timestamp_min', 'Latitude', 'Longitude', 'SOG', 'COG', 'Heading']]
    all_data = pd.concat([all_data, data], ignore_index=True)

all_data = all_data.sort_values(by='Timestamp_min')
unique_timestamps = all_data['Timestamp_min'].drop_duplicates().reset_index(drop=True)
ship_order = all_data.drop_duplicates(subset='MMSI', keep='first').sort_values('Timestamp_min')['MMSI'].tolist()
result_df = pd.DataFrame(index=unique_timestamps)

# Iterate over each vessel, build column names, and fill the data
for mmsi in ship_order:
    ship_data = all_data[all_data['MMSI'] == mmsi]
    ship_data = ship_data.set_index('Timestamp_min')
    ship_data = ship_data[['Latitude', 'Longitude', 'SOG', 'COG', 'Heading']]
    ship_columns = [f"{mmsi}_Latitude", f"{mmsi}_Longitude", f"{mmsi}_SOG", f"{mmsi}_COG", f"{mmsi}_Heading"]
    ship_data.columns = ship_columns
    result_df = result_df.join(ship_data, how='left')

result_df = result_df.reset_index(drop=False)  
result_df.insert(0, 'Timestamp_str', result_df['Timestamp_min'].dt.strftime('%Y/%m/%d %H:%M:%S'))
final_data = result_df.drop(columns=['Timestamp_min'])

# Insert two rows at the top of the result DataFrame: MMSI and column names
mmsi_row = ['']  
for mmsi in ship_order:
    mmsi_row.extend([mmsi]*5)  
    
columns_row = ['# Timestamp']  
for _ in ship_order:
    columns_row.extend(['Latitude', 'Longitude', 'SOG', 'COG', 'Heading'])
final_df = pd.DataFrame([mmsi_row, columns_row], columns=final_data.columns)
final_df = pd.concat([final_df, final_data], ignore_index=True)
output_file = "   " # Output file path
final_df.to_csv(output_file, index=False, header=False, encoding='utf-8-sig')

print(f"Data processing complete. Results saved to {output_file}")

<span style = 'color:red;font-size:25px'>Extract static attribute values

In [ ]:
summary_file = r"   "   # Summary output file path
input_folder = r"   "   # Raw data folder path
output_file = r"  "     # Output file path

summary_data = pd.read_csv(summary_file, header=None, low_memory=False)
mmsi_row = summary_data.iloc[0]
columns_row = summary_data.iloc[1]

# Get the MMSI order (drop missing values and duplicates)
mmsi_list = []
for mmsi_value in mmsi_row:
    if pd.notna(mmsi_value) and mmsi_value != '':
        try:
            mmsi = str(int(float(mmsi_value)))
            if mmsi not in mmsi_list:
                mmsi_list.append(mmsi)
        except ValueError:
            continue  

# Initialize the set of processed MMSIs
processed_mmsi = set()
mmsi_attributes = {}

# Iterate over the MMSI list and extract attribute values
for mmsi in mmsi_list:
    if mmsi in processed_mmsi:
        continue  
    found = False
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_folder, file_name)
            try:
                data = pd.read_csv(file_path)
                if 'MMSI' in data.columns and mmsi in data['MMSI'].astype(str).unique():
                    ship_data = data[data['MMSI'].astype(str) == mmsi]
                    ship_type = ship_data['Ship type'].iloc[0] if 'Ship type' in ship_data.columns else ''
                    width = ship_data['Width'].iloc[0] if 'Width' in ship_data.columns else ''
                    length = ship_data['Length'].iloc[0] if 'Length' in ship_data.columns else ''
                    draught = ship_data['Draught'].iloc[0] if 'Draught' in ship_data.columns else ''
                    mmsi_attributes[mmsi] = [ship_type, width, length, draught]
                    found = True
                    break  
            except Exception as e:
                print(f"Error reading file {file_name}: {e}")
                continue
    if not found:
        print(f"Data for MMSI {mmsi} not found; attribute values will be left empty.")
        mmsi_attributes[mmsi] = ['', '', '', '']
    processed_mmsi.add(mmsi)

# Build the result DataFrame
# Row 1: MMSI
first_row = []
for mmsi in mmsi_list:
    first_row.extend([mmsi] * 4) 

# Row 2: attribute column names
second_row = []
for _ in mmsi_list:
    second_row.extend(['Ship type', 'Width', 'Length', 'Draught'])

# Row 3: attribute values
third_row = []
for mmsi in mmsi_list:
    attributes = mmsi_attributes.get(mmsi, ['', '', '', ''])
    third_row.extend(attributes)

# Check whether lengths match
print(f"first_row length: {len(first_row)}")
print(f"second_row length: {len(second_row)}")
print(f"third_row length: {len(third_row)}")

# Create the DataFrame
num_columns = len(first_row)
result_df = pd.DataFrame(columns=range(num_columns))
result_df.loc[0] = first_row
result_df.loc[1] = second_row
result_df.loc[2] = third_row

result_df.to_csv(output_file, index=False, header=False, encoding='utf-8-sig')
print(f"Vessel attributes have been extracted and saved to {output_file}")